In [ ]:
import data_functions as dat
import filter_functions as fil
import fitting_functions as fit
import math_functions as mat
import plotting_functions as plt

import os

In [ ]:
# path to folders containing T1 data
path = r"C:\Users\Keena\Desktop\test_T1"
# extensions to look for in the folders
exts = ('.csv', '.txt')
# populate folder and file lists
folder_list, file_list = dat.dir_interogate(path, extensions=exts)
# create dictionary for indexing folders and files
folder_index = dat.make_index_dict(folder_list)
file_index = [dat.make_index_dict(file_sub_list) for file_sub_list in file_list]
# lists of the data paths and metadata (text) paths
data_path_list = dat.search_paths(folder_list, file_list, [exts[0]])
metadata_path_list = dat.search_paths(folder_list, file_list, [exts[1]])
# extract the relevant data from the files
excel_sets = [[dat.open_excel(os.path.join(path, data_path)) for data_path in data_paths] for data_paths in data_path_list]
metadata = [[dat.open_text(os.path.join(path, data_path)) for data_path in data_paths] for data_paths in metadata_path_list]
# populate dictionary for extracting from the metadata
#test = dat.create_dict(os.path.join(path, metadata_path_list[0][0]), delimiter='=')

In [ ]:
# import dictionaries used to index data
i, tp_i = dat.read_json('index_dict.json')

In [ ]:
# plot channel data to check assignment
plt.plot_scope(excel_sets[0][0][:,i['time']], [excel_sets[0][0][:,i['trig']], excel_sets[0][0][:,i['trans']], excel_sets[0][0][:,i['ref']]], multi=True)

In [ ]:
trig_indexes = [[fit.find_trigger(trig_data[:,i['trig']], modifier=0.989) for trig_data in excel_data] for excel_data in excel_sets]
tp_i['ramp'] = 15000 + trig_indexes[0][0]
# pick an offset
tp_i['r_off'] = 500
tp_i['off'] = 1000

In [ ]:
# plot transmitted and reference data to check the trigger index
plt.plot_T1_trigger(excel_sets[0][0][:,i['time']]*1e6, [excel_sets[0][0][:,i['trans']], excel_sets[0][0][:,i['ref']]], tp_i)

In [ ]:
# take a guess at the lifetimes
guess_ref_T1 = 1E-6
guess_T1 = 10E-6
# find start and stop indexes for the data (saves trying to fit to unnecesary data)
ref_stop_set = [[round(guess_ref_T1*4/(time[:,i['time']][1] - time[:,i['time']][0])) for time in time_lists] for time_lists in excel_sets]
stop_set = [[round(guess_T1*4 /(time[:,i['time']][1] - time[:,i['time']][0])) for time in time_lists] for time_lists in excel_sets]
start_set = [[value + tp_i['off'] for value in trig_index] for trig_index in trig_indexes]
ref_start_set = [[value + tp_i['r_off'] for value in trig_index] for trig_index in trig_indexes]
# create new arrays of cut data using start and stop indexes found
time_sets = [[data[:,i['time']][start:start+stop] for data, start, stop in zip(data_list, start_list, stop_lists)] for data_list, start_list, stop_lists in zip(excel_sets, start_set, stop_set)]
ref_time_sets = [[data[:,i['time']][start:start+stop] for data, start, stop in zip(data_list, start_list, stop_lists)] for data_list, start_list, stop_lists in zip(excel_sets, ref_start_set, ref_stop_set)]
trig_sets = [[data[:,i['trig']][start:start+stop] for data, start, stop in zip(data_list, start_list, stop_lists)] for data_list, start_list, stop_lists in zip(excel_sets, start_set, stop_set)]
trans_sets = [[data[:,i['trans']][start:start+stop] for data, start, stop in zip(data_list, start_list, stop_lists)] for data_list, start_list, stop_lists in zip(excel_sets, start_set, stop_set)]
ref_sets = [[data[:,i['ref']][start:start+stop] for data, start, stop in zip(data_list, start_list, stop_lists)] for data_list, start_list, stop_lists in zip(excel_sets, ref_start_set, ref_stop_set)]
# found initial start params for the fitting functions (minimum and maximum)
list_of_mins = [[min(trans) for trans in trans_lists] for trans_lists in trans_sets]
list_of_maxs = [[max(trans) for trans in trans_lists] for trans_lists in trans_sets]
ref_min_list = [[min(refs) for refs in ref_lists] for ref_lists in ref_sets]
ref_max_list = [[max(refs) for refs in ref_lists] for ref_lists in ref_sets]

In [ ]:
# zip the fit data for the reference signal
ref_zip = zip(ref_time_sets, ref_sets, ref_max_list, ref_min_list)
# fit to the reference data
ref_fit_sets = [[fit.fit_exp_decay(time, ref, params=(max_value, guess_ref_T1, min_value)) for time, ref, max_value, min_value in zip(time_set, ref_set, max_set, min_set)] for time_set, ref_set, max_set, min_set in ref_zip]
# average the fit data for each experiment
ref_fit_avg = [mat.average_arrays(ref_list) for ref_list in ref_fit_sets]

In [ ]:
# loop over references - fit and plot data of one set per folder
#for time_lists, ref_lists, fit_lists in zip(ref_time_sets, ref_sets, ref_fit_sets):
    #for time, ref, fitted in zip(time_lists, ref_lists, fit_lists):

        #plt.plot_T1_fit(time, ref, fitted[0])

In [ ]:
# define HDAWG outputs
basic_HDAWG_params = {606: 1,
                      'cp': 3,
                      'sp': 4,
                      'trig_in': 0,
                      'trig_out': 0,
                      'trig_read': 1,
                      'sequence_delay': 1E-3}

# Pit Pulse Parameters
pit_params = {'no_pulses': 280,
              'f_0': 250E6,
              'amplitude': 0.12,
              'freq_sweep': 3.5E6,
              'duration': 30E-6,
              'delay': 250E-6}

# Burn Back Parameters
burn_back_params = {'no_pulses': 100,
                    'f_0': 268.45E6,
                    'amplitude': 0.0395,
                    'freq_sweep': 1E6,
                    'duration': 48E-6,
                    'delay': 250E-6}

# AFC Pulse Params
AFC_params = {'no_pulses': 100, 
              'tau': 1E-6,
              'f_0': 268.45E6,
              'amplitude': 0.0395,
              'duration': 48E-6,
              'delay': 250E-6}

delta = 1/AFC_params['tau']
no_teeth = round((4*burn_back_params['freq_sweep'])/delta) + 2
no_holes = no_teeth - 1

AFC_params['delta'] = delta
AFC_params['no_teeth'] = no_teeth
AFC_params['no_holes'] = no_holes

# Control Pulse Params
cp_in_params = {'f_0': 250E6,
                'amplitude': 0.21,
                'duration': 0.6E-6}

cp_out_params = {'f_0': 250E6,
                 'amplitude': 0.21,
                 'duration': 0.6E-6,
                 'width': 0.3E-6}

# Signal Pulse Params
sp_params = {'f_0': 80E6,
             'amplitude': 0.21,
             'duration': 0.6E-6,
             'width': 0.3E-6}

# Heterodyne Pulse Params
hdyne_params = {'f_0': 80E6,
                'amplitude': 0.21,
                'duration': 0.6E-6,}

# Reading Pulse Params
read_params = {'f_0': 250E6,
              'amplitude': 0.015,
              'freq_sweep': 2E6,
              'duration': 30E-6}

# Shuffle Pulse Params
shuffle_params = {'f_0': 250E6,
                  'amplitude': 0.25,
                  'freq_sweep': 10.5E6,
                  'duration': 1000E-6}

# compile parameter dictionary
param_dictionary = {'HDAWG': basic_HDAWG_params,
                    'pit': pit_params,
                    'burn_back': burn_back_params,
                    'AFC': AFC_params,
                    'cp_in': cp_in_params,
                    'cp_out': cp_out_params,
                    'sp': sp_params,
                    'hdyne': hdyne_params,
                    'read': read_params,
                    'shuffle': shuffle_params,
                    'dict_one' : dict_1,
                    'dict_two': dict_2}

In [ ]:
def read_HDAWG_file(file_name, dictionary):

    with open(file_name, 'r') as file:
        awg_string = file.read()
        awg_programme = awg_string.format(**dictionary)
        
    return awg_programme

In [ ]:
read_HDAWG_file('example.txt', param_dictionary)

In [ ]:
# Dict 1
dict_1 = {'channel_1': 'on',
          'amplitude': 10}

# Dict 2
dict_2 = {'channel_2': 'on',
          'amplitude': 5}
# nested dict
nested_dict = {'dict_one' : dict_1,
               'dict_two': dict_2}

def read_file(file_name, dictionary):

    with open(file_name, 'r') as file:
        string = file.read()
        programme = string.format(**dictionary)
       
    return programme

read_file('example.txt', nested_dict)
